## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1: get and create the data frame using _pandas_

In [1]:
!pip install lxml html5lib beautifulsoup4

import pandas as pd


     |████████████████████████████████| 5.5MB 9.3MB/s eta 0:00:01
     |████████████████████████████████| 122kB 18.4MB/s eta 0:00:01


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(url)
print(len(dfs)) #outputs the tables on the page


3


In [3]:
# Get first table out of the 3                                                                                                           
df = dfs[0]

# Extract columns                                                                                                           
df2 = df[['Postal Code','Borough', 'Neighbourhood']]
#print(df2)

# make pandas dataframe:
dataframe = pd.DataFrame(df2)

# Change the column names to match the column names asked for in the assignment (note spelling!!) 
dataframe.columns =['PostalCode','Borough', 'Neighborhood'] 

# Remove all borough's that are not assigned
Dfedit = dataframe[dataframe.Borough != "Not assigned"]   #drop these rows
Dfedit.head(11)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
# Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma 

Dfedit_grouped = Dfedit.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for index, row in Dfedit_grouped.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

# Clean Notebook and add Markdown cells to explain your work and any assumptions you are making.
Dfedit_grouped

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [5]:
# use shape to print number of rows, columns.
Dfedit_grouped.shape


(103, 3)

## Part 2: utilize the Foursquare location data, create dataframe with latitude and the longitude coordinates of each neighborhood.

In [6]:
# Upload data with latitude and longitude
Geo_df = pd.read_csv('Geospatial_Coordinates.csv')
Geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#For download data, change names to match on postalcode
Geo_df.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)

# create new dataframe from others:
neighborhood = pd.merge(Dfedit_grouped, Geo_df, on='PostalCode', how='inner')
toronto_mergeddf = pd.DataFrame(neighborhood)
toronto_mergeddf

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## Part 3: Explore and cluster the neighborhoods in Toronto

In [8]:
!pip install geocoder

import geocoder # import geocoder
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library


     |████████████████████████████████| 102kB 6.5MB/s ta 0:00:011
     |████████████████████████████████| 829kB 7.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
     |████████████████████████████████| 112kB 6.0MB/s eta 0:00:01


In [9]:
# work with only boroughs that contain the word Toronto 

# use geopy to find lat and long for Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood['Latitude'], neighborhood['Longitude'], neighborhood['Borough'], neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore Toronto

In [11]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


In [20]:
# lets get the 4th Neighbourhood of Woburn

toronto_mergeddf.loc[3, 'Neighborhood']

'Woburn'

In [21]:
# we get the latitude and longitude
neighborhood_latitude = toronto_mergeddf.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_mergeddf.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_mergeddf.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Woburn are 43.7709921, -79.21691740000001.


### find the top 100 venues in Woburn:


In [23]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()


In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
#structure data and put in pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.770037,-79.221156
1,Tim Hortons,Coffee Shop,43.770827,-79.223078
2,Korean Grill House,Korean BBQ Restaurant,43.770812,-79.214502
3,"El rey del cabrito, monterrey city mexico",Mexican Restaurant,43.768800,-79.219800


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.
